# Linear and angular velocity

In [ ]:
import numpy as np
import rigid_body_motion as rbm

<div class="alert alert-info">
Note
    
The following examples require the `matplotlib` library.
</div>

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (6, 6)

Like in the previous tutorial, we first set up the world frame:

In [ ]:
rbm.register_frame("world")

For simplicity, all reference frames will share the same timestamps:

In [ ]:
n_timestamps = 10
ts = np.linspace(0, 5, n_timestamps)

We define a body moving 5 meters in the x direction:

In [ ]:
p_body_world = np.zeros((n_timestamps, 3))
p_body_world[:, 0] = np.linspace(0, 5, n_timestamps)

In addition, this body will rotate 90° around the z axis. We use helper functions from the `quaternion` package for this:

In [ ]:
from quaternion import as_float_array, from_euler_angles

o_body_world = as_float_array(
    from_euler_angles(np.linspace(0, np.pi / 4, n_timestamps), 0, 0)
)

Now we can attach a reference frame to this body:

In [ ]:
rbm.register_frame(
    "body",
    translation=p_body_world,
    rotation=o_body_world,
    timestamps=ts,
    parent="world",
)

We now define a second moving body whose motion we describe wrt the frame of the first body. It is located at 1 meter in the y direction and moves 1 meter in the negative x direction.

In [ ]:
p_body2_body = np.zeros((n_timestamps, 3))
p_body2_body[:, 0] = -np.linspace(0, 1, n_timestamps)
p_body2_body[:, 1] = 1

This body also rotates, but this time around the y axis:

In [ ]:
o_body2_body = as_float_array(
    from_euler_angles(0, np.linspace(0, np.pi / 4, n_timestamps), 0)
)

Now we can register a frame attached to the second body as a child frame of the first body frame:

In [ ]:
rbm.register_frame(
    "body2",
    translation=p_body2_body,
    rotation=o_body2_body,
    timestamps=ts,
    parent="body",
)

Let's plot the position of both bodies wrt the world frame. We use the [lookup_pose()](_generated/rigid_body_motion.lookup_pose.rst) method to obtain the position of the second body in the world frame:

In [ ]:
p_body2_world, _ = rbm.lookup_pose("body2", "world")

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

rbm.plot.points(p_body_world, ax=ax, fmt="yo-")
rbm.plot.points(p_body2_world, ax=ax, fmt="co-")
rbm.plot.reference_frame("world", ax=ax)

fig.tight_layout()

## Linear velocity

In [ ]:
v_body2_body = rbm.lookup_linear_velocity("body2", "body")
v_body2_body[0]

### Transforming the reference frame

$$v_{B2/W} = \underbrace{v_{B2/B}}_{\text{Input}} + \underbrace{v_{B/W} + \omega_{B/W} \times t_{B2/B}}_{\text{Lookup}}$$

In [ ]:
v_body2_world_t = rbm.transform_linear_velocity(
    v_body2_body, outof="body", into="world", moving_frame="body2", timestamps=ts,
)

In [ ]:
v_body2_world_l = rbm.lookup_linear_velocity("body2", "world")

In [ ]:
def compare_velocities(transform, lookup, timestamps=None, mode="linear"):
    """ Compare velocities from transform and lookup. """
    fig, axarr = plt.subplots(2, 1, sharex=True, sharey=True)

    ylabel = f"{mode.capitalize()} velocity ({'rad/s' if mode == 'angular' else 'm/s'})"

    axarr[0].plot(timestamps, transform)
    axarr[0].set_ylabel(ylabel)
    axarr[0].set_title("Transform")

    axarr[1].plot(timestamps, lookup)
    axarr[1].set_xlabel("Time (s)")
    axarr[1].set_ylabel(ylabel)
    axarr[1].set_title("Lookup")
    axarr[1].legend(["x", "y", "z"])

    fig.tight_layout()

In [ ]:
compare_velocities(v_body2_world_t, v_body2_world_l, ts)

### Transforming the moving frame

$$v_{B2/W} = \underbrace{v_{B/W}}_{\text{Input}} + \underbrace{v_{B2/B} + \omega_{B/W} \times t_{B2/B}}_{\text{Lookup}}$$

In [ ]:
v_body_world = rbm.lookup_linear_velocity("body", "world", represent_in="body")

In [ ]:
v_body2_world_t = rbm.transform_linear_velocity(
    v_body_world,
    outof="body",
    into="body2",
    reference_frame="world",
    what="moving_frame",
    timestamps=ts,
)

In [ ]:
v_body2_world_l = rbm.lookup_linear_velocity("body2", "world", represent_in="body2")

In [ ]:
compare_velocities(v_body2_world_t, v_body2_world_l, ts)

## Angular velocity

In [ ]:
w_body2_body = rbm.lookup_angular_velocity("body2", "body")
w_body2_body[0]

### Transforming the reference frame

$$\omega_{B2/W} = \underbrace{\omega_{B/W}}_{\text{Input}} + \underbrace{\omega_{B2/B}}_{\text{Lookup}}$$

In [ ]:
w_body2_world_t = rbm.transform_angular_velocity(
    w_body2_body, outof="body", into="world", timestamps=ts
)

In [ ]:
w_body2_world_l = rbm.lookup_angular_velocity("body2", "world")

In [ ]:
compare_velocities(w_body2_world_t, w_body2_world_l, ts, mode="angular")

### Transforming the moving frame

$$\omega_{B2/W} = \underbrace{\omega_{B/W}}_{\text{Input}} + \underbrace{\omega_{B2/B}}_{\text{Lookup}}$$

In [ ]:
w_body_world = rbm.lookup_angular_velocity("body", "world", represent_in="body")

In [ ]:
w_body2_world_t = rbm.transform_angular_velocity(
    w_body_world, outof="body", into="body2", what="moving_frame", timestamps=ts,
)

In [ ]:
w_body2_world_l = rbm.lookup_angular_velocity("body2", "world", represent_in="body2")

In [ ]:
compare_velocities(w_body2_world_t, w_body2_world_l, ts, mode="angular")